In [1]:
import mlflow
import os
import pandas as pd

In [2]:
os.chdir('..')

In [3]:
ls

Readme.md         image-2.png       image-6.png       requirements.txt
app/              image-3.png       image.png
data/             image-4.png       mlruns/
image-1.png       image-5.png       notebooks/


In [21]:
client = mlflow.tracking.MlflowClient()
# Cargamos experimentos que comiencen con "eval_"
experiments = [
    exp for exp in client.search_experiments() if exp.name.startswith("report_summary")
]

if not experiments:
   print("No hay experimentos disponibles.")
if experiments:
    exp_names = [exp.name for exp in experiments]
    print("Experimentos disponibles:")
    for i, exp in enumerate(exp_names):
        print(f"{i + 1}. {exp}")

Experimentos disponibles:
1. report_summary_slms_vs_gpt4_1_reference


In [22]:
experiment = client.get_experiment_by_name(exp_names[0])
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=[
        "start_time DESC",
    ],
    max_results=3500,
)

if not runs:
    print("No hay ejecuciones registradas.")
else:
    print(f"Se encontraron {len(runs)} ejecuciones registradas.")
    # Recolectamos datos de cada run
    data = []
    for run in runs:
        params = run.data.params
        metrics = run.data.metrics
        artifacts = client.list_artifacts(run.info.run_id)
        list_artifacts = [artifact for artifact in artifacts]
        dict_metrics = {
            #'run_ID': run.info.run_id,
            "video_id": params.get("video_id"),
            "channel_name": params.get("channel_name"),
            "prompt_version": params.get("prompt_version"),
            "model": params.get("llm_model"),
            # Métricas de evaluación
            "criterial_score": metrics.get("criterial_score", None),
            "embedding_cosine_distance": metrics.get("embedding_cosine_distance", None),
            "score": metrics.get("score", None),
        }
        data.append(dict_metrics)

    # Creamos un dataframe con todos los datos
    df = pd.DataFrame(data)


Se encontraron 3100 ejecuciones registradas.


In [23]:
df.model.unique().tolist()

['unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf2_Q4_k_m',
 None,
 'phi4_latest',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf3_Q4_k_m',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf3_Q8_0',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf2_Q8_0',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf_Q8_0',
 'llama3_1_8b_instruct_fp16',
 'llama3_2_3b_instruct_fp16',
 'gpt_4o_mini',
 'hf_llama_3_2_3b_finetuned_qlora_bnb_nf42_gguf_latest',
 'llama_3_2_3b_finetuned_qlora_bnb_nf42_gguf_q8_0_latest']

In [24]:
sel_models = [
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf2_Q4_k_m',
 'phi4_latest',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf3_Q4_k_m',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf3_Q8_0',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf2_Q8_0',
 'llama3_1_8b_instruct_fp16',
 'llama3_2_3b_instruct_fp16',
 'gpt_4o_mini',
 'llama_3_2_3b_finetuned_qlora_bnb_nf42_gguf_q8_0_latest']

In [38]:
df_sel = df[df['model'].isin(sel_models)]

In [39]:
# Diccionario de reemplazo de nombres de modelos
model_name_map = {
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf2_Q4_k_m':'finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q4_k_m',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf3_Q4_k_m':'finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q4_k_m',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf3_Q8_0':'finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q8_0',
 'unsloth_Meta_Llama_3_1_8B_Instruct_bnb_4bit_gguf2_Q8_0':'finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q8_0',
 'llama_3_2_3b_finetuned_qlora_bnb_nf42_gguf_q8_0_latest': 'finetune_qlora_litgpt_llama_3_2_3b_bnb_nf4_v2_q8_0',
}

# Reemplazar en la columna 'model' de df_sel
df_sel['model'] = df_sel['model'].replace(model_name_map)

/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_35880/2935142726.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['model'] = df_sel['model'].replace(model_name_map)


In [40]:
df_sel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2652 entries, 0 to 3099
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   video_id                   2652 non-null   object 
 1   channel_name               2652 non-null   object 
 2   prompt_version             2652 non-null   object 
 3   model                      2652 non-null   object 
 4   criterial_score            2652 non-null   float64
 5   embedding_cosine_distance  2652 non-null   float64
 6   score                      2652 non-null   float64
dtypes: float64(3), object(4)
memory usage: 165.8+ KB


In [41]:
import tiktoken

test_dataset_path = 'data/slm_summaries/test_slm_llama3_2_3b_instruct_fp16_v3_summary_expert.csv'
df_test = pd.read_csv(test_dataset_path,sep=";")

# Crear el codificador para llama-3.2 (usa 'cl100k_base' como aproximación)
tokenizer = tiktoken.get_encoding("cl100k_base")

# Contar tokens en la columna 'slm_summary'
df_test["slm_tokens"] = df_test["text"].apply(lambda x: len(tokenizer.encode(x)))
sel_columns =['video_id', 'channel_name','slm_tokens']
df_test_filter = df_test[sel_columns]

In [42]:
df_joined = pd.merge(df_sel, df_test_filter, on=["channel_name", "video_id"], how="inner", suffixes=('', '_test'))
df_joined['slm_tokens'].describe()

count     2652.000000
mean      7411.674208
std       7622.575768
min         38.000000
25%       2381.000000
50%       3661.000000
75%      11824.000000
max      31540.000000
Name: slm_tokens, dtype: float64

In [52]:
# Filtrar y agrupar dataset por Chunk Size y Prompt, sacar promedio del resto de columnas
# Agrupar por 'model' y 'prompt_version' y calcular estadísticas agregadas
df_grouped_1 = (
    df_sel.drop(columns=["video_id", "channel_name"])
    .groupby(["model", "prompt_version"])
    #.agg(['mean','median','std','min','max'])
    .agg(['mean'])
    .reset_index()
)

# Ordenar de mayor a menor por el score medio
df_grouped_1 = df_grouped_1.sort_values(('score', 'mean'), ascending=False)
df_grouped_1.to_csv("data/results/results_all.csv")
# Para mostrar el contenido completo del DataFrame sin truncar columnas o filas
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df_grouped_1)

,model,prompt_version,criterial_score,embedding_cosine_distance,score
,,,mean,mean,mean
5,gpt_4o_mini,v3_summary_expert,0.601810,0.100755,8.479638
4,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q8_0,v3_summary_expert,0.294118,0.159226,5.977376
2,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q8_0,v3_summary_expert,0.366516,0.159266,5.674208
3,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q4_k_m,v3_summary_expert,0.262443,0.173843,5.190045
10,phi4_latest,v1_summary_expert_one_shot,0.167421,0.185578,5.180995
1,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q4_k_m,v3_summary_expert,0.285068,0.179423,5.153846
11,phi4_latest,v3_summary_expert,0.171946,0.197469,4.506787
6,llama3_1_8b_instruct_fp16,v1_summary_expert_one_shot,0.081448,0.202654,4.325792
0,finetune_qlora_litgpt_llama_3_2_3b_bnb_nf4_v2_q8_0,v3_summary_expert,0.140271,0.224712,3.330317


In [53]:
context = 4096 # block_size=8192 or max_seq_length: 4096
# Filtrar y agrupar dataset por Chunk Size y Prompt, sacar promedio del resto de columnas
# Agrupar por 'model' y 'prompt_version' y calcular estadísticas agregadas
print(f"Filtrando por Chunk Size menor a {context}")
df_grouped_1 = (
    df_joined[df_joined["slm_tokens"]<=context].drop(columns=["video_id", "channel_name","slm_tokens"])
    .groupby(["model", "prompt_version"])
    #.agg(['mean','median','std','min','max'])
    .agg(['mean'])
    .reset_index()
)
df_grouped_1 = df_grouped_1.sort_values(('score', 'mean'), ascending=False)
df_grouped_1.to_csv(f"data/results/results_{context}.csv")
# Para mostrar el contenido completo del DataFrame sin truncar columnas o filas
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df_grouped_1)

Filtrando por Chunk Size menor a 4096


,model,prompt_version,criterial_score,embedding_cosine_distance,score
,,,mean,mean,mean
5,gpt_4o_mini,v3_summary_expert,0.685484,0.086980,8.733871
4,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q8_0,v3_summary_expert,0.387097,0.136306,6.887097
2,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q8_0,v3_summary_expert,0.516129,0.135291,6.806452
1,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q4_k_m,v3_summary_expert,0.427419,0.150576,6.314516
10,phi4_latest,v1_summary_expert_one_shot,0.290323,0.162088,6.193548
3,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q4_k_m,v3_summary_expert,0.395161,0.144866,6.177419
11,phi4_latest,v3_summary_expert,0.282258,0.167122,5.532258
6,llama3_1_8b_instruct_fp16,v1_summary_expert_one_shot,0.145161,0.180236,5.129032
0,finetune_qlora_litgpt_llama_3_2_3b_bnb_nf4_v2_q8_0,v3_summary_expert,0.193548,0.194879,4.209677


In [54]:
context = 8192 # block_size=8192 or max_seq_length: 4096, # valor maximo promedio de los ultimos años sin canal USACRIPTOMONEDAS = 6000
# Filtrar y agrupar dataset por Chunk Size y Prompt, sacar promedio del resto de columnas
print(f"Filtrando por Chunk Size menor a {context}")
# Agrupar por 'model' y 'prompt_version' y calcular estadísticas agregadas
df_grouped_1 = (
    df_joined[df_joined["slm_tokens"]<=context].drop(columns=["video_id", "channel_name","slm_tokens"])
    .groupby(["model", "prompt_version"])
     #.agg(['mean','median','std','min','max'])
    .agg(['mean'])
    .reset_index()
)
df_grouped_1 = df_grouped_1.sort_values(('score', 'mean'), ascending=False)
df_grouped_1.to_csv(f"data/results/results_{context}.csv")
# Para mostrar el contenido completo del DataFrame sin truncar columnas o filas
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df_grouped_1)

Filtrando por Chunk Size menor a 8192


,model,prompt_version,criterial_score,embedding_cosine_distance,score
,,,mean,mean,mean
5,gpt_4o_mini,v3_summary_expert,0.645570,0.090291,8.683544
4,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q8_0,v3_summary_expert,0.348101,0.146064,6.569620
2,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q8_0,v3_summary_expert,0.468354,0.144579,6.398734
1,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q4_k_m,v3_summary_expert,0.367089,0.164136,5.860759
3,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q4_k_m,v3_summary_expert,0.335443,0.157919,5.810127
10,phi4_latest,v1_summary_expert_one_shot,0.234177,0.173449,5.740506
11,phi4_latest,v3_summary_expert,0.234177,0.178007,5.177215
6,llama3_1_8b_instruct_fp16,v1_summary_expert_one_shot,0.113924,0.189635,4.822785
0,finetune_qlora_litgpt_llama_3_2_3b_bnb_nf4_v2_q8_0,v3_summary_expert,0.164557,0.203034,3.879747


In [55]:
context = 14000 # block_size=8192 or max_seq_length: 4096, # valor maximo promedio de los ultimos años sin canal USACRIPTOMONEDAS = 6000
# Filtrar y agrupar dataset por Chunk Size y Prompt, sacar promedio del resto de columnas
print(f"Filtrando por Chunk Size menor a {context}")
# Agrupar por 'model' y 'prompt_version' y calcular estadísticas agregadas
df_grouped_1 = (
    df_joined[df_joined["slm_tokens"]<=context].drop(columns=["video_id", "channel_name","slm_tokens"])
    .groupby(["model", "prompt_version"])
     #.agg(['mean','median','std','min','max'])
    .agg(['mean'])
    .reset_index()
)
df_grouped_1 = df_grouped_1.sort_values(('score', 'mean'), ascending=False)
df_grouped_1.to_csv(f"data/results/results_{context}.csv")
# Para mostrar el contenido completo del DataFrame sin truncar columnas o filas
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df_grouped_1)

Filtrando por Chunk Size menor a 14000


,model,prompt_version,criterial_score,embedding_cosine_distance,score
,,,mean,mean,mean
5,gpt_4o_mini,v3_summary_expert,0.625000,0.094007,8.653409
4,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q8_0,v3_summary_expert,0.340909,0.151314,6.397727
2,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q8_0,v3_summary_expert,0.431818,0.150443,6.232955
1,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v2_Q4_k_m,v3_summary_expert,0.335227,0.168733,5.659091
3,finetune_qlora_unsloth_llama_3_1_8B_Instruct_bnb_4bit_v3_Q4_k_m,v3_summary_expert,0.318182,0.163165,5.630682
10,phi4_latest,v1_summary_expert_one_shot,0.210227,0.177232,5.568182
11,phi4_latest,v3_summary_expert,0.215909,0.185998,4.988636
6,llama3_1_8b_instruct_fp16,v1_summary_expert_one_shot,0.102273,0.194388,4.653409
0,finetune_qlora_litgpt_llama_3_2_3b_bnb_nf4_v2_q8_0,v3_summary_expert,0.159091,0.208327,3.710227
